In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [ ]:
# --- Step 1: Define the LSTM Model ---
# This class defines our neural network architecture.
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super(LSTMClassifier, self).__init__()

        # The embedding layer converts integer-encoded words into dense vectors
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # The LSTM layer processes the sequence of word embeddings
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=n_layers,
            bidirectional=bidirectional,
            dropout=dropout,
            batch_first=True
        )
        # The final fully-connected layer maps the LSTM's output to our desired output dimension
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid() # Sigmoid for binary classification
    def forward(self, text):
        # text: [batch size, seq len]
        
        # Pass text through the embedding layer
        embedded = self.embedding(text)
        # embedded: [batch size, seq len, embedding dim]
        
        # Pass the embedded sequence through the LSTM
        # The 'h' and 'c' are the final hidden and cell states, which we don't need
        lstm_output, (hidden, cell) = self.lstm(embedded)
        # We take the final hidden state of the LSTM for classification
        # For bidirectional LSTMs, we concatenate the forward and backward final hidden states
        if self.lstm.bidirectional:
            hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        else:
            hidden = hidden[-1,:,:]
        # Pass the final hidden state through the fully-connected layer and sigmoid
        dense_output = self.fc(hidden)
        output = self.sigmoid(dense_output)
        return output

In [ ]:
# --- Step 2: Data Preprocessing ---
# A small, simple dataset for demonstration
data = [
    ("This movie was great!", 1),
    ("I loved the film.", 1),
    ("It was an awesome experience.", 1),
    ("The acting was terrible.", 0),
    ("I hated every minute of it.", 0),
    ("This film is a complete waste of time.", 0)
]